# ISM6251 - Week 3 Assignment
# Customer Data Pipeline: From Chaos to Clarity

**Student Name:** [Enter your name here]

**Student ID:** [Enter your student ID here]

**Date:** [Enter submission date]

---

## Business Context

You are a Data Analyst at **CloudTech Solutions**, a B2B SaaS company. The customer database has quality issues from multiple data sources and manual processes. Your task is to:

1. Clean and prepare customer data for churn analysis
2. Create meaningful customer segments
3. Build a reusable data preparation pipeline
4. Document data quality issues and recommendations

### Assignment Overview
- **Part 1:** Data Quality Assessment (15 points)
- **Part 2:** Missing Data Strategy (20 points)
- **Part 3:** Data Filtering & Outlier Management (20 points)
- **Part 4:** Grouping & Feature Engineering (20 points)
- **Part 5:** Data Quality Report (15 points)
- **Total: 90 points**

**Estimated Time:** 3-4 hours

---

## Part 1: Data Quality Assessment (15 points)

### Task 1.1: Setup and Data Generation

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# TODO: IMPORTANT - Replace with your actual student ID
STUDENT_ID = 'UXXX'  # <-- CHANGE THIS TO YOUR STUDENT ID

# Create filenames for your data
raw_data_filename = f"{STUDENT_ID}-week03-raw.csv"
clean_data_filename = f"{STUDENT_ID}-week03-clean.csv"

print(f"Raw data will be saved as: {raw_data_filename}")
print(f"Clean data will be saved as: {clean_data_filename}")

# Set random seed for reproducibility
np.random.seed(42)

# Configure visualization settings
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

print("\nSetup complete!")

### Task 1.2: Generate Messy Customer Data

In [ ]:
# Generate realistic messy customer data (DO NOT MODIFY THIS CELL)
def generate_messy_customer_data():
    """
    Generate a realistic messy customer dataset with various data quality issues
    """
    n_customers = 5000
    
    # Industries with typos and variations
    industries = ['Technology', 'Finance', 'Healthcare', 'Retail', 'Manufacturing', 
                 'Education', 'Real Estate', 'Consulting']
    industry_variations = {
        'Technology': ['Technology', 'Tech', 'technology', 'IT', 'Information Technology'],
        'Finance': ['Finance', 'Financial', 'Banking', 'finance', 'FinTech'],
        'Healthcare': ['Healthcare', 'Health', 'Medical', 'healthcare', 'Pharma']
    }
    
    # Contract types with inconsistencies
    contract_types = ['Monthly', 'Annual', 'Enterprise']
    contract_variations = {
        'Monthly': ['Monthly', 'monthly', 'Month-to-Month', 'MTM', '1-month'],
        'Annual': ['Annual', 'Yearly', 'annual', '12-month', '1-year'],
        'Enterprise': ['Enterprise', 'enterprise', 'Ent', 'Custom', 'ENTERPRISE']
    }
    
    data = []
    used_ids = set()
    
    for i in range(n_customers):
        # Create customer ID with some duplicates
        if i < 4950:
            customer_id = f"CUST{i+1:04d}"
        else:
            # Create 50 duplicate IDs
            customer_id = f"CUST{np.random.randint(1, 4950):04d}"
        
        # Company name (some missing)
        if np.random.random() < 0.02:
            company_name = np.nan
        else:
            company_name = f"Company_{np.random.randint(1, 3000)}"
        
        # Industry with variations and missing values
        if np.random.random() < 0.08:
            industry = np.nan
        else:
            base_industry = np.random.choice(industries)
            if base_industry in industry_variations and np.random.random() < 0.3:
                industry = np.random.choice(industry_variations[base_industry])
            else:
                industry = base_industry
        
        # Employee count with outliers and errors
        if np.random.random() < 0.05:
            employee_count = np.nan
        elif np.random.random() < 0.02:
            employee_count = np.random.choice([-10, -5, 0, 999999])  # Invalid values
        elif np.random.random() < 0.05:
            employee_count = np.random.randint(5000, 50000)  # Outliers
        else:
            employee_count = np.random.lognormal(4, 1.5)
            employee_count = int(max(1, employee_count))
        
        # Annual revenue (correlated with employees, missing for some)
        if np.random.random() < 0.175:  # 17.5% missing
            annual_revenue = np.nan
        elif employee_count and employee_count > 0:
            base_revenue = employee_count * np.random.uniform(50000, 200000)
            noise = np.random.normal(0, base_revenue * 0.2)
            annual_revenue = max(0, base_revenue + noise)
            if np.random.random() < 0.02:  # Some extreme outliers
                annual_revenue *= np.random.choice([0.01, 100])
        else:
            annual_revenue = np.nan
        
        # Signup date
        days_ago = np.random.randint(30, 1095)  # 1 month to 3 years
        signup_date = datetime.now() - timedelta(days=days_ago)
        
        # Last login date (missing indicates inactive)
        if np.random.random() < 0.125:  # 12.5% haven't logged in
            last_login_date = np.nan
        else:
            # Active users login recently, inactive users have old logins
            if np.random.random() < 0.7:  # 70% active
                days_since_login = np.random.randint(0, 30)
            else:  # 30% at risk
                days_since_login = np.random.randint(31, 180)
            last_login_date = datetime.now() - timedelta(days=days_since_login)
        
        # Monthly spend (with some negative values as errors)
        if np.random.random() < 0.06:
            monthly_spend = np.nan
        elif np.random.random() < 0.01:
            monthly_spend = np.random.uniform(-500, -10)  # Error: negative values
        else:
            if employee_count and employee_count > 0:
                base_spend = employee_count * np.random.uniform(10, 50)
                monthly_spend = max(0, base_spend + np.random.normal(0, base_spend * 0.3))
            else:
                monthly_spend = np.random.uniform(100, 5000)
        
        # Support tickets (missing for new or inactive customers)
        if np.random.random() < 0.10:  # 10% missing
            support_tickets = np.nan
        elif pd.isna(last_login_date):  # No tickets if never logged in
            support_tickets = 0
        else:
            # More tickets for problematic accounts
            support_tickets = np.random.poisson(2) if np.random.random() < 0.8 else np.random.poisson(10)
        
        # Features used (out of 20 total features)
        if np.random.random() < 0.04:
            features_used = np.nan
        elif np.random.random() < 0.01:
            features_used = np.random.choice([-1, 25, 30])  # Invalid: out of range
        else:
            features_used = np.random.binomial(20, 0.6)
        
        # Satisfaction score (1-10, with invalid values)
        if np.random.random() < 0.25:  # 25% missing
            satisfaction_score = np.nan
        elif np.random.random() < 0.01:
            satisfaction_score = np.random.choice([0, 11, 15, -5])  # Invalid scores
        else:
            # Correlated with support tickets
            if support_tickets and support_tickets > 5:
                satisfaction_score = np.random.uniform(3, 7)
            else:
                satisfaction_score = np.random.uniform(6, 10)
        
        # Contract type with variations
        if np.random.random() < 0.07:
            contract_type = np.nan
        else:
            base_contract = np.random.choice(contract_types)
            if base_contract in contract_variations and np.random.random() < 0.4:
                contract_type = np.random.choice(contract_variations[base_contract])
            else:
                contract_type = base_contract
        
        # Payment method
        if np.random.random() < 0.05:
            payment_method = np.nan
        else:
            payment_method = np.random.choice(['Credit Card', 'ACH', 'Wire Transfer', 'Invoice'])
        
        data.append({
            'customer_id': customer_id,
            'company_name': company_name,
            'industry': industry,
            'employee_count': employee_count,
            'annual_revenue': annual_revenue,
            'signup_date': signup_date,
            'last_login_date': last_login_date,
            'monthly_spend': monthly_spend,
            'support_tickets': support_tickets,
            'features_used': features_used,
            'satisfaction_score': satisfaction_score,
            'contract_type': contract_type,
            'payment_method': payment_method
        })
    
    return pd.DataFrame(data)

# Generate and save the messy data
df_raw = generate_messy_customer_data()
df_raw.to_csv(raw_data_filename, index=False)
print(f"Messy customer data generated and saved to: {raw_data_filename}")
print(f"Total records: {len(df_raw)}")
print(f"Total columns: {len(df_raw.columns)}")

### Task 1.3: Load and Explore Data

In [ ]:
# TODO: Load the raw data
df = pd.read_csv(raw_data_filename, parse_dates=['signup_date', 'last_login_date'])

print(f"Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")
print("\nFirst 5 rows:")
df.head()

In [ ]:
# TODO: Display basic information about the dataset
print("Dataset Information:")
print("="*50)
df.info()

print("\n" + "="*50)
print("Numeric Columns Summary:")
print("="*50)
df.describe()

### Task 1.4: Data Quality Assessment

In [ ]:
# TODO: Analyze missing values
missing_summary = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percent': (df.isnull().sum() / len(df)) * 100
})

missing_summary = missing_summary[missing_summary['Missing_Count'] > 0].sort_values('Missing_Percent', ascending=False)

print("Missing Value Analysis:")
print("="*50)
print(missing_summary.to_string(index=False))
print(f"\nColumns with missing values: {len(missing_summary)}/{len(df.columns)}")

In [ ]:
# TODO: Check for duplicate customer IDs
duplicate_ids = df[df.duplicated(subset=['customer_id'], keep=False)]
print(f"Duplicate customer IDs found: {len(duplicate_ids)} records")
print(f"Unique duplicate IDs: {duplicate_ids['customer_id'].nunique()}")

if len(duplicate_ids) > 0:
    print("\nSample of duplicate records:")
    print(duplicate_ids.head(10)[['customer_id', 'company_name', 'signup_date']])

In [ ]:
# TODO: Identify data quality issues
print("Data Quality Issues Identified:")
print("="*50)

# Check for negative values where they shouldn't exist
negative_employees = df[df['employee_count'] < 0]['employee_count'].count() if 'employee_count' in df.columns else 0
negative_spend = df[df['monthly_spend'] < 0]['monthly_spend'].count() if 'monthly_spend' in df.columns else 0

print(f"1. Negative employee counts: {negative_employees}")
print(f"2. Negative monthly spend: {negative_spend}")

# Check satisfaction score range
invalid_satisfaction = df[(df['satisfaction_score'] < 1) | (df['satisfaction_score'] > 10)]['satisfaction_score'].count()
print(f"3. Invalid satisfaction scores (not 1-10): {invalid_satisfaction}")

# Check features used range
invalid_features = df[(df['features_used'] < 0) | (df['features_used'] > 20)]['features_used'].count()
print(f"4. Invalid features used (not 0-20): {invalid_features}")

# Check for inconsistent categorical values
print(f"\n5. Industry value variations: {df['industry'].nunique()} unique values")
print(f"6. Contract type variations: {df['contract_type'].nunique()} unique values")

# TODO: Add your own quality checks here
# For example: Check for outliers, check date consistency, etc.

**Data Quality Observations:** 

[TODO: Write 2-3 sentences about the main data quality issues you've identified]

---

## Part 2: Missing Data Strategy (20 points)

### Task 2.1: Analyze Missing Data Patterns

In [ ]:
# TODO: Visualize missing data patterns
# Create a heatmap showing missing values
plt.figure(figsize=(12, 8))
missing_matrix = df.isnull().astype(int)
sns.heatmap(missing_matrix, cmap='RdYlBu', cbar_kws={'label': 'Missing (1) vs Present (0)'}, yticklabels=False)
plt.title('Missing Data Pattern Visualization')
plt.xlabel('Features')
plt.ylabel('Customers')
plt.tight_layout()
plt.show()

# Analyze correlations between missing values
missing_corr = missing_matrix.corr()
print("\nCorrelation between missing values:")
print("(High correlation suggests data is not missing completely at random)")
print(missing_corr.loc[missing_corr.abs().gt(0.3) & (missing_corr != 1)].dropna(how='all', axis=0).dropna(how='all', axis=1))

### Task 2.2: Decide Drop vs Impute Strategy

In [ ]:
# TODO: Make decisions based on missing percentage and business importance
def classify_missing_strategy(column, missing_pct, df):
    """
    Classify the strategy for handling missing data based on the guidelines from lecture
    """
    if missing_pct > 50:
        return "DROP_COLUMN"
    elif missing_pct < 5:
        return "DROP_ROWS"
    elif 10 <= missing_pct <= 50:
        # Important columns to impute
        important_cols = ['annual_revenue', 'monthly_spend', 'satisfaction_score', 'support_tickets']
        if column in important_cols:
            return "IMPUTE"
        else:
            return "IMPUTE"  # Default to impute for moderate missing
    else:  # 5-10% gray area
        return "IMPUTE"  # Conservative approach

# Create strategy table
strategy_df = pd.DataFrame({
    'Column': df.columns,
    'Missing_Percent': (df.isnull().sum() / len(df)) * 100
})

strategy_df['Strategy'] = strategy_df.apply(
    lambda row: classify_missing_strategy(row['Column'], row['Missing_Percent'], df), axis=1
)

print("Missing Data Strategy:")
print("="*60)
for strategy in ['DROP_COLUMN', 'DROP_ROWS', 'IMPUTE']:
    cols = strategy_df[strategy_df['Strategy'] == strategy]['Column'].tolist()
    if cols:
        print(f"\n{strategy}:")
        for col in cols:
            pct = strategy_df[strategy_df['Column'] == col]['Missing_Percent'].values[0]
            print(f"  - {col}: {pct:.1f}% missing")

### Task 2.3: Implement Imputation

In [ ]:
# TODO: Implement your imputation strategy
from sklearn.impute import SimpleImputer

# Create a copy for cleaning
df_clean = df.copy()

# Step 1: Handle duplicates first
print("Handling duplicate customer IDs...")
# Keep the most recent record for each customer_id
df_clean = df_clean.sort_values('signup_date').drop_duplicates(subset=['customer_id'], keep='last')
print(f"Removed {len(df) - len(df_clean)} duplicate records")

# Step 2: Drop columns with >50% missing (if any)
cols_to_drop = strategy_df[strategy_df['Strategy'] == 'DROP_COLUMN']['Column'].tolist()
if cols_to_drop:
    print(f"\nDropping columns with >50% missing: {cols_to_drop}")
    df_clean = df_clean.drop(columns=cols_to_drop)

# Step 3: Impute numeric columns
numeric_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df_clean.select_dtypes(include=['object']).columns.tolist()

print("\nImputing numeric columns...")
for col in numeric_cols:
    if df_clean[col].isnull().sum() > 0:
        # Check skewness to decide between mean and median
        if df_clean[col].skew() > 1:
            # Use median for skewed data
            imputer = SimpleImputer(strategy='median')
            df_clean[col] = imputer.fit_transform(df_clean[[col]]).ravel()
            print(f"  - {col}: Imputed with median (skewed distribution)")
        else:
            # Use mean for normal-ish data
            imputer = SimpleImputer(strategy='mean')
            df_clean[col] = imputer.fit_transform(df_clean[[col]]).ravel()
            print(f"  - {col}: Imputed with mean")

# Step 4: Impute categorical columns
print("\nImputing categorical columns...")
for col in categorical_cols:
    if col != 'customer_id' and df_clean[col].isnull().sum() > 0:
        # Use mode for categorical
        imputer = SimpleImputer(strategy='most_frequent')
        df_clean[col] = imputer.fit_transform(df_clean[[col]]).ravel()
        print(f"  - {col}: Imputed with mode (most frequent value)")

# Verify no missing values remain in critical columns
remaining_missing = df_clean.isnull().sum()
if remaining_missing.sum() > 0:
    print("\nRemaining missing values:")
    print(remaining_missing[remaining_missing > 0])
else:
    print("\nAll missing values handled successfully!")

---

## Part 3: Data Filtering & Outlier Management (20 points)

### Task 3.1: Fix Invalid Values

In [ ]:
# TODO: Fix invalid values based on business rules
print("Fixing invalid values...")
print("="*50)

# Fix negative employee counts
invalid_employees = df_clean['employee_count'] <= 0
if invalid_employees.sum() > 0:
    df_clean.loc[invalid_employees, 'employee_count'] = df_clean['employee_count'].median()
    print(f"Fixed {invalid_employees.sum()} invalid employee counts (replaced with median)")

# Fix negative monthly spend
invalid_spend = df_clean['monthly_spend'] < 0
if invalid_spend.sum() > 0:
    df_clean.loc[invalid_spend, 'monthly_spend'] = df_clean['monthly_spend'].median()
    print(f"Fixed {invalid_spend.sum()} negative monthly spend values")

# Fix satisfaction scores outside 1-10 range
invalid_satisfaction = (df_clean['satisfaction_score'] < 1) | (df_clean['satisfaction_score'] > 10)
if invalid_satisfaction.sum() > 0:
    # Clip to valid range
    df_clean['satisfaction_score'] = df_clean['satisfaction_score'].clip(1, 10)
    print(f"Fixed {invalid_satisfaction.sum()} invalid satisfaction scores (clipped to 1-10)")

# Fix features_used outside 0-20 range
invalid_features = (df_clean['features_used'] < 0) | (df_clean['features_used'] > 20)
if invalid_features.sum() > 0:
    df_clean['features_used'] = df_clean['features_used'].clip(0, 20)
    print(f"Fixed {invalid_features.sum()} invalid features_used values (clipped to 0-20)")

# Standardize categorical values
print("\nStandardizing categorical values...")

# Standardize industry names
industry_mapping = {
    'Tech': 'Technology', 'technology': 'Technology', 'IT': 'Technology', 
    'Information Technology': 'Technology',
    'Financial': 'Finance', 'Banking': 'Finance', 'finance': 'Finance', 'FinTech': 'Finance',
    'Health': 'Healthcare', 'Medical': 'Healthcare', 'healthcare': 'Healthcare', 'Pharma': 'Healthcare'
}
df_clean['industry'] = df_clean['industry'].replace(industry_mapping)
print(f"  - Industry: Reduced from {df['industry'].nunique()} to {df_clean['industry'].nunique()} categories")

# Standardize contract types
contract_mapping = {
    'monthly': 'Monthly', 'Month-to-Month': 'Monthly', 'MTM': 'Monthly', '1-month': 'Monthly',
    'Yearly': 'Annual', 'annual': 'Annual', '12-month': 'Annual', '1-year': 'Annual',
    'enterprise': 'Enterprise', 'Ent': 'Enterprise', 'Custom': 'Enterprise', 'ENTERPRISE': 'Enterprise'
}
df_clean['contract_type'] = df_clean['contract_type'].replace(contract_mapping)
print(f"  - Contract Type: Reduced from {df['contract_type'].nunique()} to {df_clean['contract_type'].nunique()} categories")

### Task 3.2: Detect and Handle Outliers

In [ ]:
# TODO: Detect outliers using IQR method
def detect_outliers_iqr(df, column, multiplier=1.5):
    """Detect outliers using IQR method"""
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Check for outliers in key numeric columns
outlier_columns = ['employee_count', 'annual_revenue', 'monthly_spend', 'support_tickets']

print("Outlier Detection Results:")
print("="*50)

outlier_summary = {}
for col in outlier_columns:
    outliers, lower, upper = detect_outliers_iqr(df_clean, col, multiplier=3)  # Using 3*IQR for conservative approach
    outlier_summary[col] = {
        'count': len(outliers),
        'percentage': (len(outliers) / len(df_clean)) * 100,
        'lower_bound': lower,
        'upper_bound': upper
    }
    print(f"\n{col}:")
    print(f"  - Outliers: {len(outliers)} ({(len(outliers)/len(df_clean))*100:.1f}%)")
    print(f"  - Bounds: [{lower:.0f}, {upper:.0f}]")

In [ ]:
# TODO: Visualize outliers
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.ravel()

for idx, col in enumerate(outlier_columns):
    axes[idx].boxplot(df_clean[col].dropna())
    axes[idx].set_title(f'{col}\n({outlier_summary[col]["count"]} outliers)')
    axes[idx].set_ylabel('Value')
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Outlier Detection - Box Plots', y=1.02, fontsize=14)
plt.show()

In [ ]:
# TODO: Handle outliers based on business logic
print("Handling outliers based on business logic...")
print("="*50)

# For this assignment, we'll cap extreme outliers rather than remove them
# This preserves the data while reducing their impact

for col in outlier_columns:
    # Use 99th percentile as cap for upper outliers
    upper_cap = df_clean[col].quantile(0.99)
    lower_cap = df_clean[col].quantile(0.01)
    
    original_outliers = ((df_clean[col] > upper_cap) | (df_clean[col] < lower_cap)).sum()
    
    # Cap the values
    df_clean[col] = df_clean[col].clip(lower_cap, upper_cap)
    
    print(f"{col}: Capped {original_outliers} extreme values to 1st-99th percentile range")

print("\nOutlier handling complete!")

---

## Part 4: Grouping & Feature Engineering (20 points)

### Task 4.1: Create Customer Segments

In [ ]:
# TODO: Create meaningful customer segments based on multiple criteria

# First, create some useful features
df_clean['days_since_login'] = (pd.Timestamp.now() - df_clean['last_login_date']).dt.days
df_clean['account_age_days'] = (pd.Timestamp.now() - df_clean['signup_date']).dt.days
df_clean['revenue_per_employee'] = df_clean['annual_revenue'] / df_clean['employee_count']
df_clean['avg_ticket_per_month'] = df_clean['support_tickets'] / (df_clean['account_age_days'] / 30)

# Define customer segments based on value and engagement
def segment_customer(row):
    """Segment customers based on value and engagement"""
    
    # High value threshold (top 30%)
    high_value_threshold = df_clean['monthly_spend'].quantile(0.70)
    
    # Engagement based on login recency
    if pd.isna(row['days_since_login']) or row['days_since_login'] > 90:
        engagement = 'Inactive'
    elif row['days_since_login'] <= 30:
        engagement = 'Active'
    else:
        engagement = 'At-Risk'
    
    # Value based on monthly spend
    if row['monthly_spend'] >= high_value_threshold:
        value = 'High-Value'
    elif row['monthly_spend'] >= df_clean['monthly_spend'].quantile(0.30):
        value = 'Standard'
    else:
        value = 'Low-Value'
    
    # Combine for final segment
    if value == 'High-Value' and engagement == 'Active':
        return 'Champions'
    elif value == 'High-Value' and engagement == 'At-Risk':
        return 'At-Risk High-Value'
    elif value == 'High-Value' and engagement == 'Inactive':
        return 'Lost High-Value'
    elif value == 'Standard' and engagement == 'Active':
        return 'Loyal Customers'
    elif value == 'Standard' and engagement == 'At-Risk':
        return 'Need Attention'
    elif value == 'Low-Value' and engagement == 'Active':
        return 'Promising'
    else:
        return 'Low Priority'

# Apply segmentation
df_clean['customer_segment'] = df_clean.apply(segment_customer, axis=1)

# Display segment distribution
segment_distribution = df_clean['customer_segment'].value_counts()
print("Customer Segment Distribution:")
print("="*50)
for segment, count in segment_distribution.items():
    percentage = (count / len(df_clean)) * 100
    print(f"{segment}: {count} customers ({percentage:.1f}%)")

### Task 4.2: Analyze Segments

In [ ]:
# TODO: Create segment profiles using groupby operations
segment_profiles = df_clean.groupby('customer_segment').agg({
    'customer_id': 'count',
    'monthly_spend': 'mean',
    'annual_revenue': 'mean',
    'employee_count': 'mean',
    'satisfaction_score': 'mean',
    'support_tickets': 'mean',
    'features_used': 'mean',
    'days_since_login': 'mean',
    'account_age_days': 'mean'
}).round(1)

segment_profiles.rename(columns={'customer_id': 'count'}, inplace=True)

print("Segment Profiles:")
print("="*80)
print(segment_profiles.to_string())

In [ ]:
# TODO: Answer key business questions using groupby
print("Key Business Insights:")
print("="*50)

# Question 1: Which industry has the highest average monthly spend?
industry_spend = df_clean.groupby('industry')['monthly_spend'].agg(['mean', 'count']).sort_values('mean', ascending=False)
print("\n1. Top Industries by Average Monthly Spend:")
print(industry_spend.head())

# Question 2: What's the relationship between contract type and satisfaction?
contract_satisfaction = df_clean.groupby('contract_type').agg({
    'satisfaction_score': 'mean',
    'customer_id': 'count',
    'monthly_spend': 'mean'
}).round(2)
print("\n2. Contract Type Analysis:")
print(contract_satisfaction)

# Question 3: Which payment method is most popular among high-value customers?
high_value_customers = df_clean[df_clean['customer_segment'].str.contains('High-Value')]
payment_preference = high_value_customers['payment_method'].value_counts()
print("\n3. Payment Method Preference (High-Value Customers):")
print(payment_preference)

# Question 4: Feature usage by segment
feature_usage = df_clean.groupby('customer_segment')['features_used'].mean().sort_values(ascending=False)
print("\n4. Average Features Used by Segment:")
print(feature_usage.round(1))

### Task 4.3: Create Additional Features

In [ ]:
# TODO: Engineer additional features that might be useful for churn prediction

# Engagement score (0-100)
df_clean['engagement_score'] = (
    (df_clean['features_used'] / 20) * 40 +  # 40 points for feature usage
    ((30 - df_clean['days_since_login'].clip(0, 30)) / 30) * 30 +  # 30 points for recency
    (df_clean['satisfaction_score'] / 10) * 30  # 30 points for satisfaction
)

# Support intensity (tickets per month)
df_clean['support_intensity'] = df_clean['support_tickets'] / (df_clean['account_age_days'] / 30)

# Value tier
df_clean['value_tier'] = pd.qcut(df_clean['monthly_spend'], q=3, labels=['Low', 'Medium', 'High'])

# Churn risk flag (simplified)
df_clean['churn_risk'] = (
    (df_clean['days_since_login'] > 60) | 
    (df_clean['satisfaction_score'] < 5) |
    (df_clean['support_intensity'] > 3)
).astype(int)

print("New features created:")
print("="*50)
print("1. engagement_score: Combined metric of user engagement (0-100)")
print("2. support_intensity: Support tickets per month")
print("3. value_tier: Customer value category (Low/Medium/High)")
print("4. churn_risk: Binary flag for potential churn risk")

print("\nChurn Risk Summary:")
churn_summary = df_clean['churn_risk'].value_counts()
print(f"At Risk: {churn_summary.get(1, 0)} customers ({churn_summary.get(1, 0)/len(df_clean)*100:.1f}%)")
print(f"Stable: {churn_summary.get(0, 0)} customers ({churn_summary.get(0, 0)/len(df_clean)*100:.1f}%)")

---

## Part 5: Data Quality Report (15 points)

### Task 5.1: Summary Statistics

In [ ]:
# TODO: Create comprehensive data quality report
print("="*60)
print("DATA QUALITY REPORT - CloudTech Solutions Customer Data")
print("="*60)
print(f"Report Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}")
print(f"Analyst: {STUDENT_ID}")
print()

print("1. DATA OVERVIEW")
print("-" * 40)
print(f"Original Records: {len(df)}")
print(f"Clean Records: {len(df_clean)}")
print(f"Records Removed: {len(df) - len(df_clean)} ({(len(df) - len(df_clean))/len(df)*100:.1f}%)")
print(f"Features: {len(df_clean.columns)} (added {len(df_clean.columns) - len(df.columns)} engineered features)")
print()

print("2. DATA QUALITY ISSUES ADDRESSED")
print("-" * 40)
print(f"✓ Duplicate customer IDs: {len(df[df.duplicated(subset=['customer_id'], keep=False)])} → 0")
print(f"✓ Missing values: {df.isnull().sum().sum()} → {df_clean.isnull().sum().sum()}")
print(f"✓ Invalid employee counts: {(df['employee_count'] <= 0).sum()} → 0")
print(f"✓ Invalid satisfaction scores: {((df['satisfaction_score'] < 1) | (df['satisfaction_score'] > 10)).sum()} → 0")
print(f"✓ Industry variations: {df['industry'].nunique()} → {df_clean['industry'].nunique()}")
print(f"✓ Contract type variations: {df['contract_type'].nunique()} → {df_clean['contract_type'].nunique()}")
print()

print("3. IMPUTATION SUMMARY")
print("-" * 40)
imputed_cols = ['annual_revenue', 'satisfaction_score', 'support_tickets', 'monthly_spend']
for col in imputed_cols:
    original_missing = df[col].isnull().sum()
    if original_missing > 0:
        print(f"{col}: {original_missing} values imputed ({original_missing/len(df)*100:.1f}%)")
print()

print("4. CUSTOMER SEGMENTATION")
print("-" * 40)
for segment in df_clean['customer_segment'].value_counts().head(5).index:
    count = (df_clean['customer_segment'] == segment).sum()
    avg_spend = df_clean[df_clean['customer_segment'] == segment]['monthly_spend'].mean()
    print(f"{segment}: {count} customers (${avg_spend:,.0f} avg monthly)")
print()

print("5. KEY METRICS")
print("-" * 40)
print(f"Average Monthly Spend: ${df_clean['monthly_spend'].mean():,.2f}")
print(f"Average Satisfaction Score: {df_clean['satisfaction_score'].mean():.2f}/10")
print(f"Customers at Churn Risk: {df_clean['churn_risk'].sum()} ({df_clean['churn_risk'].mean()*100:.1f}%)")
print(f"Average Engagement Score: {df_clean['engagement_score'].mean():.1f}/100")

### Task 5.2: Save Clean Data

In [ ]:
# TODO: Save the cleaned dataset
df_clean.to_csv(clean_data_filename, index=False)
print(f"Clean data saved to: {clean_data_filename}")
print(f"File contains {len(df_clean)} rows and {len(df_clean.columns)} columns")
print("\nDataset is now ready for machine learning!")

### Task 5.3: Recommendations

## Data Quality Recommendations

Based on my analysis of CloudTech Solutions' customer data, I recommend the following:

### Immediate Actions:

1. **Data Collection Improvements:**
   - [TODO: Add your recommendation about preventing duplicate customer IDs]
   - [TODO: Add your recommendation about reducing missing satisfaction scores]

2. **High-Priority Customer Segments:**
   - [TODO: Which segment needs immediate attention and why?]
   - [TODO: What action should be taken for at-risk high-value customers?]

3. **Data Quality Monitoring:**
   - [TODO: What metric should be monitored regularly?]
   - [TODO: What threshold would indicate data quality problems?]

### Long-term Improvements:

1. **Systematic Data Governance:**
   - [TODO: One suggestion for improving data entry processes]
   
2. **Feature Engineering for Churn Model:**
   - [TODO: What additional data would improve churn prediction?]

### Key Insights:

[TODO: Write 2-3 sentences summarizing the most important findings from your analysis]


---

## BONUS: Advanced Analysis (Optional, +10 points)

### Bonus Task 1: Advanced Imputation (+3 points)
Implement KNN or Iterative imputation for better handling of missing values

In [ ]:
# BONUS: Implement advanced imputation
# TODO: Use KNNImputer or IterativeImputer from sklearn
# Compare results with simple imputation

### Bonus Task 2: Data Quality Visualization (+3 points)
Create visualizations showing data quality improvements

In [ ]:
# BONUS: Create before/after visualizations
# TODO: Show the impact of your data cleaning process visually

### Bonus Task 3: Reusable Pipeline (+4 points)
Create a reusable function that encapsulates your entire data cleaning process

In [ ]:
# BONUS: Create a reusable data preparation pipeline
def prepare_customer_data(df_raw):
    """
    Complete data preparation pipeline for customer data
    
    Parameters:
    df_raw: Raw customer dataframe
    
    Returns:
    df_clean: Cleaned and feature-engineered dataframe
    report: Dictionary containing quality metrics
    """
    # TODO: Implement your complete pipeline here
    pass

---

## Submission Checklist

Before submitting:
- [ ] Student name and ID filled in
- [ ] All code cells run without errors
- [ ] Data quality issues identified and documented
- [ ] Missing data strategy implemented
- [ ] Outliers handled appropriately
- [ ] Customer segments created
- [ ] Data quality report completed
- [ ] Recommendations provided
- [ ] Clean CSV file saved with correct name

**Files to submit:**
1. This notebook (.ipynb)
2. HTML export of notebook
3. Your cleaned CSV data file

**Great job completing your data preparation pipeline!**